In [1]:
import sys
# %pip install -q stellargraph[demos]==1.2.1

In [2]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

2022-02-08 15:34:33.053739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-08 15:34:33.053757: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-08 15:34:34.187811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-08 15:34:34.187828: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-08 15:34:34.187842: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (oana-Inspiron-5593): /proc/driver/nvidia/version does not exist
2022-02-08 15:34:34.188470: I tensorflow/core/platform/cpu

In [3]:
# %pip install pyvis
from pyvis import network as net
import networkx as nx
from stellargraph import StellarGraph
import pandas as pd

In [13]:
square_weight_edges = pd.DataFrame(
    {"source": ["a", "b", "c", "d", "a", "b", "c", "d", "a", "d"], "target": ["b", "c", "d", "a", "c", "a", "b", "d", "a", "c"], "weight": [1.0, 0.2, 3.4, 5.67, 1.0, 1.0, 1.0, 1.0, 1.0,1.0]}
)
square_weight_edges

,source,target,weight
0,a,b,1.00
1,b,c,0.20
2,c,d,3.40
3,d,a,5.67
4,a,c,1.00
5,b,a,1.00
6,c,b,1.00
7,d,d,1.00
8,a,a,1.00
9,d,c,1.00


In [14]:
square_node_data = pd.DataFrame(
    {"x": [1, 2, 3, 4], "y": [-0.2, 0.3, 0.0, -0.5]}, index=["a", "b", "c", "d"]
)
square_node_data

,x,y
a,1,-0.2
b,2,0.3
c,3,0.0
d,4,-0.5


In [15]:
G = StellarGraph(
{"action": square_node_data}, {"line": square_weight_edges}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 4, Edges: 10

 Node types:
  action: [4]
    Features: float32 vector, length 2
    Edge types: action-line->action

 Edge types:
    action-line->action: [10]
        Weights: range=[0.2, 5.67], mean=1.627, std=1.64241
        Features: none


In [16]:
g_1 = nx.Graph()
g_1 = nx.from_pandas_edgelist(square_weight_edges, 'source', 'target')

nt = net.Network(notebook=True)
nt.from_nx(g_1)
nt.show('nx.html')

In [17]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 1 positive and 1 negative edges. **


In [18]:
print(edge_ids_test, edge_labels_test)

[['d' 'd']
 ['d' 'b']] [1 0]


In [19]:
G_test.info()

'StellarGraph: Undirected multigraph\n Nodes: 4, Edges: 9\n\n Node types:\n  action: [4]\n    Features: float32 vector, length 2\n    Edge types: action-line->action\n\n Edge types:\n    action-line->action: [9]\n        Weights: range=[0.2, 5.67], mean=1.69667, std=1.7263\n        Features: none'

In [26]:
G_test.edges()

[('a', 'b'),
 ('a', 'b'),
 ('a', 'd'),
 ('a', 'c'),
 ('a', 'a'),
 ('b', 'c'),
 ('b', 'c'),
 ('c', 'd'),
 ('c', 'd')]

In [27]:
G.edges()

[('a', 'b'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'a'),
 ('a', 'c'),
 ('b', 'a'),
 ('c', 'b'),
 ('d', 'd'),
 ('a', 'a'),
 ('d', 'c')]

In [60]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, _ = dataset.load(subject_as_feature=True)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 1440
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)
        Features: none


In [73]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

print(G_test.info())

print(G_test.node_ids_to_ilocs([edge_ids_test[0][0],edge_ids_test[0][1]]))

** Sampled 542 positive and 542 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 4887

 Node types:
  paper: [2708]
    Features: float32 vector, length 1440
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [4887]
        Weights: all 1 (default)
        Features: none

[22563 36140]
[ 431 2137]


In [51]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

print(G_train.info())


** Sampled 488 positive and 488 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 4399

 Node types:
  paper: [2708]
    Features: float32 vector, length 1440
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [4399]
        Weights: all 1 (default)
        Features: none


# Common Neighbors

In [108]:
from sknetwork.data import house
from sknetwork.linkpred import *

In [9]:
adjacency = house()

In [21]:
print(adjacency)

  (0, 1)	True
  (0, 4)	True
  (1, 0)	True
  (1, 2)	True
  (1, 4)	True
  (2, 1)	True
  (2, 3)	True
  (3, 2)	True
  (3, 4)	True
  (4, 0)	True
  (4, 1)	True
  (4, 3)	True


In [22]:
cn = CommonNeighbors()
similarities = cn.fit_predict(adjacency, 1)
similarities

array([1, 3, 0, 2, 1])

In [23]:
cn.predict([0, 1, 2, 3 ,4])

array([[2, 1, 1, 1, 1],
       [1, 3, 0, 2, 1],
       [1, 0, 2, 0, 2],
       [1, 2, 0, 2, 0],
       [1, 1, 2, 0, 3]])

In [32]:
adjacency_cora = G_test.to_adjacency_matrix()
adjacency_cora[0]

<1x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 5 stored elements in Compressed Sparse Row format>

In [131]:
# cn = CommonNeighbors()
# cn = JaccardIndex()
# cn = SaltonIndex() #higher than jaccard
# cn = SorensenIndex()
cn = HubPromotedIndex() #high on cora test
# cn = HubDepressedIndex()
# cn = AdamicAdar() #very high, some error 
# cn = ResourceAllocation()
similarities = cn.fit_predict(adjacency_cora, 0)
similarities = cn.predict(list(range(adjacency_cora.shape[0])))
similarities

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [132]:
from collections import Counter
maximum = max(max(x) for x in similarities)
one_list = [y for x in similarities for y in x]
print(maximum) #Jaccard between 0 and 1

print(Counter(one_list).most_common()[:10])
# threshold = 3 # if 8 common neighbours or more ->connected
threshold = 0.5

1.0
[(0.0, 7250644), (0.5, 27154), (0.3333333432674408, 19098), (1.0, 14068), (0.25, 10854), (0.20000000298023224, 4704), (0.1666666716337204, 1708), (0.6666666865348816, 1616), (0.4000000059604645, 668), (0.1428571492433548, 538)]


In [133]:
from sklearn.metrics import accuracy_score

nodes_test = [list(G_test.node_ids_to_ilocs([edge_ids_test[i][0],edge_ids_test[i][1]])) for i in range(len(edge_ids_test))]
# print(nodes_test)

predicted = []
for (node1, node2), label in zip(nodes_test, edge_labels_test):
#     print(node1, node2, label)
    common_neighbour_similarity = cn.predict((node1, node2))
    result = 0 if common_neighbour_similarity < threshold else 1
    predicted.append(result)
#     print(common_neighbour_similarity)

accuracy = accuracy_score(edge_labels_test, predicted)
print(accuracy)

0.6291512915129152
